In [1]:
# import modules
import pandas as pd
from IPython.display import display

**<span style="color:crimson">1. Naive data linkage without blocking</span>**

In [2]:
# read csv files
google_products = pd.read_csv('google_small.csv')
amazon_products = pd.read_csv('amazon_small.csv')
true = pd.read_csv('amazon_google_truth_small.csv').sort_values(by='idAmazon')

In [3]:
# create jaccard similarity function
def get_jaccard_sim(str1, str2):
    a = set(str1.split())
    b = set(str2.split())
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))

In [4]:
# set a common column for merge
google_products['key'] = 1
amazon_products['key'] = 1

In [5]:
# join the 2 dataframes
joined = pd.merge(google_products, amazon_products, on='key').drop('key', axis=1)
# calculate the scores for the names/titles using the jaccard index
joined['name_score'] =joined.apply(lambda row: get_jaccard_sim(row['name'], row['title']), axis=1)
# calculate the scores for the price similarities
joined['price_score'] = joined.apply(lambda row: (min(row['price_x'], row['price_y'])/max(row['price_x'], row['price_y'])), axis=1)
# calculate the final scores with the price weighted less as there is more chance of duplicates
joined['final_score'] = joined['name_score'] + joined['price_score']/2

In [6]:
# threshold determined through trial and error, only concerned with values above this threshold
THRESHOLD = 0.54
joined = joined[joined['final_score'] > THRESHOLD]
# take only the largest score for each amazonID for comparison
joined = joined.sort_values(by='final_score', ascending=False).drop_duplicates(['idAmazon'])
# create new dataframes for faster calculations
predicted = joined.loc[:, ['idAmazon', 'idGoogleBase']].sort_values(by='idAmazon')

In [7]:
# create a dataframe of tp values
tp_values = []
tp_df = predicted.merge(true, on=['idAmazon', 'idGoogleBase'])
tp_df.head()

,idAmazon,idGoogleBase
0,1931102953,http://www.google.com/base/feeds/snippets/1272...
1,b00002s6sc,http://www.google.com/base/feeds/snippets/1049...
2,b00004nhn7,http://www.google.com/base/feeds/snippets/1843...
3,b000051sgq,http://www.google.com/base/feeds/snippets/1758...
4,b000067fk7,http://www.google.com/base/feeds/snippets/3785...


In [8]:
# calculate precision and recall using tp, fp, fn
tp = len(tp_df)
fp = len(predicted) - tp
fn = len(true) - tp
precision = tp/(tp+fp)
recall = tp/(tp+fn)
print(f'precision = {precision}')
print(f'recall = {recall}')

precision = 0.9147286821705426
recall = 0.9076923076923077


<span style="color:green">**DISCUSSION**</span><br>
After testing, it was decided that manufacturer and description should not be used in linkage as this lead to too much variance in results. 

A jaccard index, which is used to measure the overlap of two strings, was used to compare the strings in order to get a score based on the similarity of the titles, this was used as titles/names will usually be quite similar across platforms and the jaccard index will usually lead to accurate results while measuring short titles such as the ones in these datasets.

to calculate the similarity of the prices, i took the minimum value of the two and divided that by the maximum value of the two, this leads to creating a (smaller than one) score based on how much smaller the first number is from the second.

The final score was decided by summing the name score and half the price score. The reason half the price score was used was due to it being a less accurate representation of similarity (multiple items can have the same/similar price) in comparison to name which will very rarely have the same/similar values.

The threshold for determining the scores was done through trial and error in order to get the best balance between precision and recall. We also only accounted for the idAmazon's with the highest final scores when comparing with our truth dataset as there can only be one true match for each id, this sufficienty improved the performance of our linkage.

The performance shows us precision ~ 0.915 and recall ~ 0.908. These values both appear to be very good as we have a very high rate of correct linkage between our datasets while still covering a large amount of the true values. 

**<span style="color:crimson">1. Blocking for efficient data linkage</span>**

In [9]:
# create dataframes from datasets
google_products = pd.read_csv('google.csv')
amazon_products = pd.read_csv('amazon.csv')
true = pd.read_csv('amazon_google_truth.csv')
true = true.rename(columns={'idGoogleBase':'id'})

In [10]:
# preprocess price data
# assume prices are in gbp already so no need to apply exchange rate
google_products['price'] = google_products['price'].str.replace('gbp', '')
amazon_products['price'] = amazon_products['price'].replace('gbp', '')
google_products['price'] = pd.to_numeric(google_products['price'])
amazon_products['price'] = pd.to_numeric(amazon_products['price'])
amazon_products['price'].max()

101515.55

In [11]:
# create blocks and labels for price and allocate each id to a block
blocks = [0, 20, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 100000, 1000000]
labels = list(range(1,12))
amazon_products['bin'] = pd.cut(amazon_products.price, bins=blocks, include_lowest=True, right=True, labels=labels)
google_products['bin'] = pd.cut(google_products.price, bins=blocks, include_lowest=True, right=True, labels=labels)

In [12]:
# create a common column to merge on
google_products['key'] = 1
amazon_products['key'] = 1

matches = []
fp = 0

# find matches within the ground truth and each block
for i in labels:
    amazon = amazon_products.loc[amazon_products['bin'] == i]
    google = google_products.loc[google_products['bin'] == i]
    joined = pd.merge(google, amazon, on='key').drop('key', axis=1).loc[:, ['id', 'idAmazon']]
    df = joined.merge(true, on=['idAmazon', 'id'])
    fp += (len(joined) - len(df))
    matches.append(df)
                
matches = pd.concat(matches)
matches.head()

,id,idAmazon
0,http://www.google.com/base/feeds/snippets/1056...,b000aoz7hw
1,http://www.google.com/base/feeds/snippets/1290...,b000hcl5sm
2,http://www.google.com/base/feeds/snippets/1823...,b000jx1kgq
3,http://www.google.com/base/feeds/snippets/1826...,b000h25yr0
4,http://www.google.com/base/feeds/snippets/1825...,b0009yegpc


In [13]:
# calculate PC and RR
tp = len(matches)
fn = len(true) - tp
PC = tp/(tp+fn)
n = len(amazon_products)*len(google_products)
RR = 1-(tp+fp)/n
print(f'Pair completeness = {PC}')
print(f'Reduction ratio = {RR}')

Pair completeness = 0.7876923076923077
Reduction ratio = 0.8019180184478734


<span style="color:green">**DISCUSSION**</span><br>

It was decided to use a blocking method based on price ranges. This was decided by somewhat of a trial and error process, as well as deduction, I originally attempted to block with n-grams on title but this lead to having far too many blocks and turned out to be a poor way to block these datasets. The sizes of the price blocks were determined by prior knowledge of how items are generally priced and the largest block size was determined by calling .max() on the prices of both dataframes. I also decided to not impute the 0 values of price in the amazon.csv file to anything, this is because imputing it to the mean/median would be improper as we are solely blocking on price and there is large variance in prices.

This blocking method seemed to work quite well, both the PC and RR are around 80% which is quite high. These results are not as good as the results produced from the measures in the naive-linkage implemented earlier but this is to be expected as blocking is generally a less precise method as it prioritises speed and security over results. If a naive linkage method was used here, it would be very slow as there are millions of possible combinations between the datasets, using blocking significantly sped up the process.

pair completeness measures the coverage of our values from within the truth dataset, with our value being ~80%, we have quite a good recall considering that blocking was used this could potentially be increased further by adding a second conjunction to the blocking scheme. Reduction ratio is a measure of the effectiveness of data reduction, here we also have a value of ~80% which means that we have a high ratio of data ingested to data matched.
